#### The dataset used here is day_9_test_new.txt , and should be renamed as Test_day_9.txt in order to run the code

# Preprocessing of data and Isolation Forest
Based on what they did here: https://github.com/pnnl/safekit/blob/master/safekit/features/lanl/word_feats.py . Create a Dictionary of Words containing the Tokens as 'keys' and their integer representations as 'value'.

In [1]:
import time
import pandas as pd
# dataset_train = pd.read_csv("Training_day_9.txt", sep = ",", na_values = ['?','.'])
dataset_test = pd.read_csv("Test_day_9.txt", sep = ",", na_values = ['?','.'])

In [2]:
# dataset_train = dataset_train.drop('Unnamed: 0',1)
# dataset_test = dataset_test.drop('Unnamed: 0',1)
# dataset_train = dataset_train.drop('time',1)
dataset_test = dataset_test.drop('time',1)
dataset_test.rename(columns={'anomaly':'Malignant/Benign'}, inplace=True)

In [3]:
data = dataset_test

In [4]:
data.head()

,source_user@domain,destination_user@domain,source_computer,destination_computer,authentication_type,logon_type,authentication_orientation,success,Malignant/Benign
0,U1653@DOM1,U1653@DOM1,C17693,C395,NTLM,Network,LogOn,Success,1
1,U2164@DOM1,U2164@DOM1,C6438,C2382,Kerberos,Network,LogOn,Success,0
2,U7988@DOM1,U7988@DOM1,C2039,C457,Kerberos,Network,LogOn,Success,0
3,C1708$@DOM1,C1708$@DOM1,C1709,C528,Kerberos,Network,LogOn,Success,0
4,C11388$@DOM1,C11388$@DOM1,C11388,C529,Kerberos,Network,LogOn,Success,0


In [5]:
A = data # will be resused for the confusion matrix
data = data.drop('Malignant/Benign', 1) # because the model is Unsupervised

### Data processing using factorization

In [6]:
data.iloc[:,0] = pd.factorize(data.iloc[:,0])[0]
data.iloc[:,1] = pd.factorize(data.iloc[:,1])[0]
data.iloc[:,2] = pd.factorize(data.iloc[:,2])[0]
data.iloc[:,3] = pd.factorize(data.iloc[:,3])[0]
data.iloc[:,4] = pd.factorize(data.iloc[:,4])[0]
data.iloc[:,5] = pd.factorize(data.iloc[:,5])[0]
data.iloc[:,6] = pd.factorize(data.iloc[:,6])[0]
data.iloc[:,7] = pd.factorize(data.iloc[:,7])[0]

In [7]:
data.head()

,source_user@domain,destination_user@domain,source_computer,destination_computer,authentication_type,logon_type,authentication_orientation,success
0,0,0,0,0,0,0,0,0
1,1,1,1,1,1,0,0,0
2,2,2,2,2,1,0,0,0
3,3,3,3,3,1,0,0,0
4,4,4,4,4,1,0,0,0


### Perform anomaly detection on resulting processed dataset using IsolationForest
Thanks to Manasa for her model which this is based on.

In [8]:
from sklearn.ensemble import  IsolationForest 

# Add label for validation
data_y = data
data_y['anomaly'] = A['Malignant/Benign']

iforest = IsolationForest(n_estimators=50, contamination=0.14, bootstrap = True)

start_time = time.time()

iforest = iforest.fit(data)
pred_isoF = iforest.predict(data)

stop_time = time.time()
print("Prediction run in %.2fs" % (stop_time - start_time))

Prediction run in 1.05s


In [9]:
pred_isoF

array([-1,  1,  1, ..., -1,  1,  1])

In [10]:

data['pred'] = pred_isoF


In [11]:
import numpy as np

# -1 is outlier, i.e. what we hope to be the anomaly
TP = np.sum((data_y['anomaly'] == 1) & (data['pred'] == -1))
FN = np.sum((data_y['anomaly'] == 1) & (data['pred'] ==  1))
FP = np.sum((data_y['anomaly'] == 0) & (data['pred'] == -1))
TN = np.sum((data_y['anomaly'] == 0) & (data['pred'] ==  1))

print ("TP: ", TP,"\nFN: ",  FN, "\nFP: ",  FP, "\nTN: ",  TN)

# Performance Matrix
accuracy = (TP+TN)/(TP+FN+FP+TN)
print ("Accuracy: ", np.round(accuracy, 2))
recall = TP/(TP+FN)
print ("recall: ", np.round(recall, 2))
specificity = TN/(TN+FP)
print ("Specificity: ", np.round(specificity, 2))
precision = TP/(TP + FP)
print ("precision: ", np.round(precision, 2))
F1 = 2/((1/precision) + (1/recall))
print ("F1: ", np.round(F1, 2))

TP:  200 
FN:  0 
FP:  1228 
TN:  8772
Accuracy:  0.88
recall:  1.0
Specificity:  0.88
precision:  0.14
F1:  0.25
